In [1]:
%pip install rdflib
%pip install owlrl
%pip install pandas
from rdflib import Graph, Namespace
import pandas as pd

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pandas in c:\users\agavr\appdata\local\programs\python\python310\lib\site-packages (1.5.1)



In [2]:
# Define namespaces
sphn = Namespace("https://biomedit.ch/rdf/sphn-ontology/sphn#")
sulo = Namespace("https://w3id.org/sulo/")
snomed = Namespace("http://snomed.info/id/")
aidava = Namespace("https://rdf.aidava.eu/resource/")

# Load SPHN RDF file
g = Graph()
g.parse("sphn_administrative_case.ttl", format="turtle")
g.bind("sphn", sphn)
g.bind("sulo", sulo)
g.bind("snomed", snomed)
g.bind("aidava", aidava)
print(f"{len(g)} Triples")

24 Triples


In [3]:
# Create a new graph for transformed data
# Define SPARQL transformation query
# This graph contains blank node structures that miss specific variable bindings

transformation_query = """
PREFIX sphn: <https://biomedit.ch/rdf/sphn-ontology/sphn#>
PREFIX sulo: <https://w3id.org/sulo/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX snomed: <http://snomed.info/id/>
PREFIX : <https://rdf.aidava.eu/resource/>

CONSTRUCT {
  ?case a sphn:AdministrativeCase .
  ?case sulo:hasPart [ a snomed:305056002;  sulo:isLocatedIn ?originLocation ;sulo:occursIn [ a sulo:Time; sulo:hasValue ?admissionDateTime ] ].
  ?case sulo:hasPart [ a snomed:58000006;  sulo:isLocatedIn ?dischargeLocation ; sulo:occursIn [ a sulo:Time; sulo:hasValue ?dischargeDateTime ] ].
  ?case sulo:isLocatedIn ?location.
  ?case sulo:hasParticipant [ a :PatientRole; sulo:isFeatureOf ?subjectPseudoIdentifier ].
}

WHERE {
  OPTIONAL { ?case sphn:hasAdmissionDateTime ?admissionDateTime. }
  OPTIONAL { ?case sphn:hasOriginLocation ?originLocation. }

  OPTIONAL { ?case sphn:hasDischargeDateTime ?dischargeDateTime. }
  OPTIONAL { ?case sphn:hasLocation ?location. }

  OPTIONAL { ?case sphn:hasDischargeLocation ?dischargeLocation. }
  OPTIONAL { ?case sphn:hasSubjectPseudoIdentifier ?subjectPseudoIdentifier. }
}
"""

# Create a new graph for transformed data
transformed_g = g.query(transformation_query)
sulo_graph = transformed_g.graph
sulo_graph.bind("sphn", sphn)
sulo_graph.bind("sulo", sulo)
sulo_graph.bind("snomed", snomed)
sulo_graph.bind("aidava", aidava)

sulo_graph.serialize(destination="transformed_sulo.ttl", format='turtle')

<Graph identifier=N58fd3f374cd84c3d87267a68f6ce9616 (<class 'rdflib.graph.Graph'>)>

## Transform the RDF in SPHN to SULO

In [4]:
### TESTING

# Define SPARQL transformation query
transformation_query = """
PREFIX sphn: <https://biomedit.ch/rdf/sphn-ontology/sphn#>
PREFIX sulo: <https://w3id.org/sulo/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX snomed: <http://snomed.info/id/>
PREFIX : <https://rdf.aidava.eu/resource/>

CONSTRUCT {
  ?case a sphn:AdministrativeCase .
  ?case sulo:hasPart ?admission .

  # ?admission a snomed:305056002 .
  # ?admission sulo:isLocatedIn ?originLocation .
  # ?admission sulo:occursIn ?admission_time .
  # ?admission_time a sulo:Time .
  # ?admission_time sulo:hasValue ?admissionDateTime .

  # ?case sulo:hasPart [ a snomed:305056002;  sulo:isLocatedIn ?originLocation ;sulo:occursIn [ a sulo:Time; sulo:hasValue ?admissionDateTime ] ].
  # ?case sulo:hasPart [ a snomed:58000006;  sulo:isLocatedIn ?dischargeLocation ; sulo:occursIn [ a sulo:Time; sulo:hasValue ?dischargeDateTime ] ].
  # ?case sulo:isLocatedIn ?location.
  # ?case sulo:hasParticipant [ a :PatientRole; sulo:isFeatureOf ?subjectPseudoIdentifier ].
}

WHERE {
  ?case a sphn:AdministrativeCase .
  BIND( IF( BOUND(?admissionDateTime) || BOUND(?originLocation) ,
      UUID(), # then: generate a UUID
       ""     # else: return a fallback string
    )
    AS ?uuid

   OPTIONAL {
    FILTER(BOUND(?admissionDateTime))
    BIND(IRI(CONCAT(STR(?case), "admission",STR(?admissionDateTime))) AS ?admission)
    #BIND(IRI(CONCAT(STR(?admission), "admission_time")) AS ?admission_time)
  }

  OPTIONAL { ?case sphn:hasAdmissionDateTime ?admissionDateTime. }
  OPTIONAL { ?case sphn:hasOriginLocation ?originLocation. }

  # OPTIONAL { ?case sphn:hasDischargeDateTime ?dischargeDateTime. }
  # OPTIONAL { ?case sphn:hasLocation ?location. }

  # OPTIONAL { ?case sphn:hasDischargeLocation ?dischargeLocation. }
  # OPTIONAL { ?case sphn:hasSubjectPseudoIdentifier ?subjectPseudoIdentifier. }
}
"""

# Create a new graph for transformed data
transformed_g = g.query(transformation_query)
sulo_graph = transformed_g.graph
sulo_graph.bind("sphn", sphn)
sulo_graph.bind("sulo", sulo)
sulo_graph.bind("snomed", snomed)
sulo_graph.bind("aidava", aidava)

sulo_graph.serialize(destination="transformed_sulo.ttl", format='turtle')

ParseException: Expected {SelectQuery | ConstructQuery | DescribeQuery | AskQuery}, found 'BIND'  (at char 1013), (line:26, col:3)

In [5]:
# Define SPARQL query
query_x = """
PREFIX sphn: <https://biomedit.ch/rdf/sphn-ontology/sphn#>
PREFIX sulo: <https://w3id.org/sulo/>
PREFIX snomed: <http://snomed.info/id/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?case ?admissionDateTime
WHERE {
   ?case a sphn:AdministrativeCase .
   ?case sulo:hasPart ?admission_process .
   ?admission_process a snomed:305056002 .
   ?admission_process sulo:occursIn ?admissionDateTime .
}
"""

In [6]:
transformed_g = Graph()
transformed_g.parse("transformed_sulo.ttl", format="turtle")

<Graph identifier=N33f9c25de0ad47eb8748a0c4cecef3a1 (<class 'rdflib.graph.Graph'>)>

In [7]:
# Run query on transformed graph
results = transformed_g.query(query_x)

# Print results
for row in results:
    print(f"Visit: {row}")

Visit: (rdflib.term.URIRef('https://rdf.aidava.eu/resource/AdministrativeCase/2604'), rdflib.term.BNode('n334b77d4b96840bf873a020ef9aae1bfb4'))


In [8]:
sulo2sphnv2025 = """
PREFIX sphn: <https://biomedit.ch/rdf/sphn-ontology/sphn#>
PREFIX sulo: <https://w3id.org/sulo/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX snomed: <http://snomed.info/id/>
PREFIX : <https://rdf.aidava.eu/resource/>

CONSTRUCT {
  ?case a sphn:AdministrativeCase .

  ?case sphn:admission ?admission .
  ?admission a sphn:Admission.
  ?admission sphn:hasDateTime ?admissionDateTime .
  ?admission sphn:hasOriginLocation ?originLocation .

  ?case sphn:discharge ?discarge .
  ?discarge a sphn:Discharge .
  ?discarge sphn:hasDateTime ?dischargeDateTime .
  ?discarge sphn:hasTargetLocation ?targetLocation .
}
WHERE {
    ?case a sphn:AdministrativeCase .
    OPTIONAL {
      ?case sulo:hasPart ?admission .
      ?admission a snomed:305056002 .
      OPTIONAL {
        ?admission sulo:occursIn ?time .
        ?time a sulo:Time .
        ?time sulo:hasValue ?admissionDateTime .
      }
      OPTIONAL {
        ?admission sulo:isLocatedIn ?originLocation .
      }
    }
    OPTIONAL {
      ?case sulo:hasPart ?discharge .
      ?discharge a snomed:58000006 .
      OPTIONAL {
        ?discharge sulo:occursIn ?time .
        ?time a sulo:Time .
        ?time sulo:hasValue ?dischargeDateTime .
      }
      OPTIONAL {
        ?discharge sulo:isLocatedIn ?targetLocation .
      }
    }


}
"""

results = transformed_g.query(sulo2sphnv2025)
graph = results.graph
graph.bind("sphn", sphn)
graph.bind("sulo", sulo)
graph.bind("snomed", snomed)
graph.bind("aidava", aidava)
graph.serialize(destination="sulo2sphnv2025.ttl", format='turtle')


<Graph identifier=N6a9790325b7145d49b003c06e609a263 (<class 'rdflib.graph.Graph'>)>